In [ ]:
import spikeinterface.full as si

In [ ]:
%matplotlib widget

### Use simulated data for now

In [ ]:
rec, sort = si.generate_ground_truth_recording(durations=[60], num_channels=128, num_units=50)

In [ ]:
si.plot_probe_map(rec)

In [ ]:
si.plot_traces(rec, backend="ipywidgets")

### Extract waveforms

Start with simulated data

In [ ]:
we = si.extract_waveforms(rec, sort, folder=None, mode="memory", n_jobs=-1)

### Get spike scalings

In [ ]:
amp_scalings = si.compute_amplitude_scalings(we)

## Get residuals

In [ ]:
all_templates = we.get_all_templates()

In [ ]:
residuals_w_scaling = si.InjectTemplatesRecording(
    sorting=sort,
    templates=-all_templates,
    amplitude_factor=amp_scalings,
    parent_recording=rec
)
residuals_wout_scaling = si.InjectTemplatesRecording(
    sorting=sort,
    templates=-all_templates,
    amplitude_factor=None,
    parent_recording=rec
)

In [ ]:
rec_dict = dict(
    original=rec,
    residuals_w=residuals_w_scaling,
    residuals_wout=residuals_wout_scaling
)

In [ ]:
si.plot_traces(rec_dict, backend="ipywidgets")

# Stream NWB

### Load IBL recording

In [ ]:
rec_ibl = si.read_nwb_recording(
    file_path="https://api.dandiarchive.org/api/assets/b2fee6b4-6049-4720-8d3d-44f8ec483527/download/",
    stream_mode="remfile",
    electrical_series_path="acquisition/ElectricalSeriesAp00"
)

In [ ]:
rec_f = si.highpass_filter(rec_ibl)
rec_de = si.highpass_spatial_filter(rec_f)
rec_cmr = si.common_reference(rec_f)

In [ ]:
rec_dict = dict(
    raw=rec_ibl,
    hp=rec_f,
    destripe=rec_de,
    cmr=rec_cmr
)

In [ ]:
si.plot_traces(rec_dict, backend="ipywidgets")

### Save 10 s data

In [ ]:
rec_ibl_cut = rec_ibl.frame_slice(0, int(10 * rec_ibl.sampling_frequency))

In [ ]:
rec_ibl_saved = rec_ib_cut.save(folder="ibl_snippet_10", n_jobs=-1)

In [ ]:
rec_f = si.highpass_filter(rec_ibl_cut)
rec_de = si.highpass_spatial_filter(rec_f)
rec_cmr = si.common_reference(rec_f)